# Machine interpreter

## Instructions

Three registers, `a`, `b`, and `c`. 
Program counter, `pc`.
Unlimited memory in numbered addresses, completely separate from program storage.

Each register and memory location can hold 8-byte integers (Java `long`s).

Machine carries out the instruction at location `pc`. After it's executed, `pc` increments by 1.

`jmp` and `jpz` override this normal change in `pc`.

| Instruction | Description |
|:------------|:------------|
| `inc r`     | increment contents of register `r` |
| `dec r`     | decrement contents of register `r` |
| `set r i`   | set contents of register `r` to literal value `i` |
| `cpy r s`   | set contents of register `r` into register `s` | 
| `sto r m`   | store contents of register `r` into memory location `m` |
| `ld r m`    | load contents of memory location `m` into register `r` |
| `jmp i`     | jump to instruction `i` places forward |
| `jpz r i`   | jump to instruction `i` places forward if<br>register `r` contains zero, otherwise continue to next instruction |

For `jmp` and `jpz`, `i` is relative to the current instruction. `i` can be negative to jump to earlier places in the program. `i`=1 is a no-op, `i`=0 causes an infinite loop.

In [22]:
def new_machine():
    return {'pc': 0, 
            'a': 0,
            'b': 0, 
            'c': 0,
            'instructions': []}

In [23]:
def show_machine(machine):
    return ', '.join('{}: {}'.format(sk, machine[int(sk) if sk.isnumeric() else sk]) 
                     for sk in sorted(str(k) for k in machine)
                     if sk != 'instructions')

In [24]:
def inc(reg, machine):
    machine[reg] += 1
    machine['pc'] += 1

In [25]:
def dec(reg, machine):
    machine[reg] -= 1
    machine['pc'] += 1

In [26]:
def jmp(addr, machine):
    machine['pc'] += addr

In [27]:
def jpz(reg, addr, machine):
    if machine[reg] == 0:
        machine['pc'] += addr
    else:
        machine['pc'] += 1

In [28]:
def set_literal(reg, literal, machine):
    machine[reg] = literal
    machine['pc'] += 1

In [29]:
def cpy(from_reg, to_reg, machine):
    machine[to_reg] = machine[from_reg]
    machine['pc'] += 1

In [30]:
def sto(reg, addr, machine):
    machine[addr] = machine[reg]
    machine['pc'] += 1

In [31]:
def ld(reg, addr, machine):
    if addr in machine:
        machine[reg] = machine[addr]
    else:
        machine[reg] = 0
    machine['pc'] += 1

In [32]:
instruction_table = {'inc': inc, 'dec': dec, 'jmp': jmp,
                    'jpz': jpz, 'set': set_literal, 'cpy': cpy,
                    'sto': sto, 'ld': ld}
numeric_args_table = {'jmp': [0], 'jpz': [1], 'set': [1], 'sto': [1], 'ld': [1]}

In [33]:
def parse(instruction):
    words = instruction.split()
    instr = words[0]
    args = words[1:]
    if instr in numeric_args_table:
        for p in numeric_args_table[instr]:
            args[p] = int(args[p])
    return instruction_table[instr], args

In [34]:
m = new_machine()
inc('a', m)
cargs = ['a', 'b']
cpy(*cargs, m)
inc('a', m)
m

{'a': 2, 'b': 1, 'c': 0, 'instructions': [], 'pc': 3}

In [35]:
def program_from_instructions(prog, machine):
    machine['instructions'] = [parse(instr) for instr in prog]

In [36]:
def program_from_listing(listing, machine):
    labelled_instructions = [i for i in listing.split('\n') if i]
    instructions = replace_labels(labelled_instructions)
    program_from_instructions(instructions, machine)

In [37]:
def replace_labels(listing):
    locations = {}
    for n, i in enumerate(listing):
        if ':' in i:
            locations[i.split(':')[0]] = n

    unlabelled_listing = []
    for n, i in enumerate(listing):
        instr = i.split()
        if ':' in i:
            instr = i.split(':')[1].split()
        else:
            instr = i.split()
        terms = []
        for term in instr:
            if term in locations:
                terms += [str(locations[term] - n)]
            else:
                terms += [term]
        transformed_instr = ' '.join(terms)
        unlabelled_listing += [transformed_instr]
        
    return unlabelled_listing    

In [38]:
'fred: inc a'.split(':')[1].split()

['inc', 'a']

In [39]:
program = """
      set a 10
loop: dec a
      inc b
      sto b 1
      jpz a 2
      jmp loop
"""
labelled_instructions = [i for i in program.split('\n') if i]
instructions = replace_labels(labelled_instructions)
instructions

['set a 10', 'dec a', 'inc b', 'sto b 1', 'jpz a 2', 'jmp -4']

In [40]:
def run(machine, initial_state=None, trace=False):
    if initial_state:
        machine.update(initial_state)
    while machine['pc'] < len(machine['instructions']):
        if trace:
            print(show_machine(machine))
        cmd, args = machine['instructions'][machine['pc']]
        cmd(*args, machine)

In [41]:
def execute(listing, initial_state=None, trace=False):
    m = new_machine()
    program_from_listing(listing, m)
    run(m, initial_state=initial_state, trace=trace)
    return m

In [42]:
program = """
inc a
inc a
cpy a b
inc a
"""
execute(program)

{'a': 3,
 'b': 2,
 'c': 0,
 'instructions': [(<function __main__.inc>, ['a']),
  (<function __main__.inc>, ['a']),
  (<function __main__.cpy>, ['a', 'b']),
  (<function __main__.inc>, ['a'])],
 'pc': 4}

In [43]:
program = """
set a 10
dec a
inc b
sto b 1
jpz a 2
jmp -4
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 20})

{'b': 10,
 1: 10,
 'instructions': [(<function __main__.set_literal>, ['a', 10]),
  (<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.sto>, ['b', 1]),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-4])],
 'pc': 6,
 'a': 0,
 'c': 20}

In [44]:
program = """
      set a 10
loop: dec a
      inc b
      sto b 1
      jpz a 2
      jmp loop
"""
execute(program, initial_state={'c': 20})

{'b': 10,
 1: 10,
 'instructions': [(<function __main__.set_literal>, ['a', 10]),
  (<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.sto>, ['b', 1]),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-4])],
 'pc': 6,
 'a': 0,
 'c': 20}

In [45]:
program = """
cpy c a
set b 0
dec a
jpz b 3
dec b
jmp 2
inc b
jpz a 3
jmp -6
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 5})

{'a': 0,
 'b': 1,
 'c': 5,
 'instructions': [(<function __main__.cpy>, ['c', 'a']),
  (<function __main__.set_literal>, ['b', 0]),
  (<function __main__.dec>, ['a']),
  (<function __main__.jpz>, ['b', 3]),
  (<function __main__.dec>, ['b']),
  (<function __main__.jmp>, [2]),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 3]),
  (<function __main__.jmp>, [-6])],
 'pc': 10}

In [49]:
# b holds parity of number in c: (c % 2)
program = """
      cpy c a
      set b 0
loop: dec a
      jpz b odd
      dec b
      jmp end
odd:  inc b
end:  jpz a 2
      jmp loop
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 5})

{'a': 0,
 'b': 1,
 'c': 5,
 'instructions': [(<function __main__.cpy>, ['c', 'a']),
  (<function __main__.set_literal>, ['b', 0]),
  (<function __main__.dec>, ['a']),
  (<function __main__.jpz>, ['b', 3]),
  (<function __main__.dec>, ['b']),
  (<function __main__.jmp>, [2]),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-6])],
 'pc': 9}

In [52]:
# c holds floor(a/2)
program = """
      sto a 10
      set c 0
      set b 0
loop: dec a
      jpz b odd
      dec b
      inc c
      jmp end
odd:  inc b
end:  jpz a 2
      jmp loop
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
show_machine(execute(program, initial_state={'a': 7}))

'10: 7, a: 0, b: 1, c: 3, pc: 11'